Reset and prepare

In [35]:
USE master;
GO
DROP DATABASE IF EXISTS [dfu23e_transactions];
GO
CREATE DATABASE [dfu23e_transactions];
GO
USE [dfu23e_transactions];

Query was canceled by user

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.348

Setup tables

In [8]:
CREATE TABLE Accounts (
    AccountID INT PRIMARY KEY IDENTITY(1,1),
    AccountNumber NVARCHAR(20) UNIQUE NOT NULL,
    AccountHolderName NVARCHAR(100) NOT NULL,
    Balance DECIMAL(18, 2) NOT NULL
);
CREATE TABLE Transactions (
    TransactionID INT PRIMARY KEY IDENTITY(1,1),
    SourceAccount NVARCHAR(20) NOT NULL,
    DestinationAccount NVARCHAR(20) NOT NULL,
    Amount DECIMAL(18, 2) NOT NULL,
    TransactionDate DATETIME NOT NULL
);
GO

INSERT INTO Accounts (AccountNumber, AccountHolderName, Balance)
VALUES ('A', 'John Doe', 1000.00);

INSERT INTO Accounts (AccountNumber, AccountHolderName, Balance)
VALUES ('B', 'Jane Smith', 500.00);


Commands completed successfully.

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.034

Transaction example

In [8]:
SELECT * FROM Accounts;

BEGIN TRANSACTION;

UPDATE Accounts
SET Balance = Balance - 100
WHERE AccountNumber = 'A';

UPDATE Accounts
SET Balance = BAlance + 100
WHERE AccountNumber = 'B';

INSERT INTO Transactions (SourceAccount, DestinationAccount, Amount, TransactionDate)
VALUES ('A', 'B', 100, GETDATE());

COMMIT TRANSACTION;

SELECT * FROM Accounts;

SELECT * FROM Transactions;

(2 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(2 rows affected)

(4 rows affected)

Total execution time: 00:00:00.014

AccountID,AccountNumber,AccountHolderName,Balance
1,A,John Doe,700.00
2,B,Jane Smith,800.00


AccountID,AccountNumber,AccountHolderName,Balance
1,A,John Doe,600.00
2,B,Jane Smith,900.00


TransactionID,SourceAccount,DestinationAccount,Amount,TransactionDate
1,A,B,100.00,2023-09-12 18:40:01.713
2,A,B,100.00,2023-09-12 18:40:25.160
3,A,B,100.00,2023-09-12 18:40:41.403
4,A,B,100.00,2023-09-12 18:40:59.040


Prepare example with error

In [32]:
UPDATE Accounts SET Balance = 110 WHERE AccountNumber = 'A';

(1 row affected)

Total execution time: 00:00:00.008

Example, with possible error

In [34]:
SELECT * FROM Accounts;

BEGIN TRANSACTION;

PRINT 'Deduct money from account A'
DECLARE @SourceAccountBalance DECIMAL(18, 2);
SELECT @SourceAccountBalance = Balance FROM Accounts WHERE AccountNumber = 'A';

PRINT 'Check if there is enough balance in account A'
IF @SourceAccountBalance >= 100
BEGIN
    UPDATE Accounts
    SET Balance = Balance - 100
    WHERE AccountNumber = 'A';

    PRINT 'Add money to account B'
    UPDATE Accounts
    SET Balance = Balance + 100
    WHERE AccountNumber = 'B';

    PRINT 'Insert a record into the transactions table to track the transaction'
    INSERT INTO Transactions (SourceAccount, DestinationAccount, Amount, TransactionDate)
    VALUES ('A', 'B', 100, GETDATE());

    COMMIT TRANSACTION;
    PRINT 'SUCCESS: Check if everything is fine, and if so, commit the transaction'
END
ELSE
BEGIN
    ROLLBACK TRANSACTION;
    PRINT 'ERROR: Transaction rolled back due to insufficient balance in Account A.';
END;

SELECT * FROM Accounts;

SELECT * FROM Transactions;

(2 rows affected)

Deduct money from account A

Check if there is enough balance in account A

ERROR: Transaction rolled back due to insufficient balance in Account A.

(2 rows affected)

(13 rows affected)

Total execution time: 00:00:00.003

AccountID,AccountNumber,AccountHolderName,Balance
1,A,John Doe,10.00
2,B,Jane Smith,1800.00


AccountID,AccountNumber,AccountHolderName,Balance
1,A,John Doe,10.00
2,B,Jane Smith,1800.00


TransactionID,SourceAccount,DestinationAccount,Amount,TransactionDate
1,A,B,100.00,2023-09-12 18:43:47.133
2,A,B,100.00,2023-09-12 18:44:07.333
3,A,B,100.00,2023-09-12 18:44:13.450
4,A,B,100.00,2023-09-12 18:44:15.673
5,A,B,100.00,2023-09-12 18:44:26.320
6,A,B,100.00,2023-09-12 18:44:27.267
7,A,B,100.00,2023-09-12 18:44:28.093
8,A,B,100.00,2023-09-12 18:44:28.950
9,A,B,100.00,2023-09-12 18:44:29.587
10,A,B,100.00,2023-09-12 18:44:30.323
